# Phase 3-2: Training and Testing for $t=2$
* Models : SVM, MLP, LightGBM, XGBoost, CatBoost

## Setup

First, let's import a few common modules.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
import random
seed_val = 43
np.random.seed(seed_val)
random.seed(seed_val)


## Data Loading

In [2]:
seed = 42
df = pd.read_csv(f"datasets/train_set_{seed}_t_2.csv")

In [3]:
X = df.drop(['is_drop'], axis=1)
y = df['is_drop'].copy()

## Data Transformation

Label encoding for categorical features

In [4]:
def sex_mapping(sex):
    sex_map = {'M': 0, 'F': 1}
    return sex_map[sex]

def seme_mapping(seme): # semester (Spring/Fall)
    seme_map = {1: 0, 2: 1}
    return seme_map[seme]

def college_mapping(college):
    college_map = {'TH': 0, 'HS': 1, 'BZ': 2, 'HT': 3, 'EG': 4, 'SW': 5, 'AT': 6}
    return college_map[college]


In [5]:
def label_encoding(data):
    data['sex'] = data['sex'].map(sex_mapping)
    data['semester'] = data['semester'].map(seme_mapping)
    data['college'] = data['college'].map(college_mapping)

In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_attrs = ['grade', 'gpa_last_seme', 'credits_last_seme', 'credits_tot', 'n_seme', 'years_since']
cat_attrs = ['semester', 'sex', 'adm_unit', 'nation', 'in_capa', 'college', 'leave']

full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', OneHotEncoder(), cat_attrs)
])

cat_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', 'passthrough', cat_attrs)
])

X_t = full_pipeline.fit_transform(X)    # for SVM and MLP with one-hot encoding
X_cat = X.copy()    # for CatBoost without encoding
label_encoding(X)   # for XGBoost and LightGBM with label encoding
X_xgb = cat_pipeline.fit_transform(X)
X_cat = cat_pipeline.fit_transform(X) # for CatBoost

X = pd.DataFrame(X_t, columns=full_pipeline.get_feature_names_out())        # for SVM and MLP
X_xgb = pd.DataFrame(X_xgb, columns=cat_pipeline.get_feature_names_out())   # for XGBoost and LightGBM
X_cat = pd.DataFrame(X_cat, columns=cat_pipeline.get_feature_names_out())   # for CatBoost
cat_indices = [6, 7, 8, 9, 10, 11, 12]


In [7]:
# Typesetting for LGBM
num_cols = ['num__grade', 'num__gpa_last_seme', 'num__credits_last_seme', 'num__credits_tot', 'num__n_seme', 'num__years_since']
cat_cols = list(set(X_cat.columns) - set(num_cols))

X_cat[num_cols] = X_cat[num_cols].astype(float)
X_cat[cat_cols] = X_cat[cat_cols].astype(int)
X_xgb[num_cols] = X_xgb[num_cols].astype(float)
X_xgb[cat_cols] = X_xgb[cat_cols].astype(int)

In [8]:
X_cat.rename(columns={'num__grade': 'year', 'num__gpa_last_seme': 'gpa_last', 'num__credits_last_seme': 'credits_last',
                  'num__credits_tot': 'credits_tot', 'num__n_seme': 'n_semesters', 'num__years_since': 'years_since',
                  'cat__semester': 'release', 'cat__sex': 'sex', 'cat__adm_unit': 'adm_type', 'cat__nation': 'nation',
                  'cat__in_capa': 'in_capa', 'cat__college': 'college', 'cat__leave': 'status'}, inplace=True)

## 5-fold CV and Threshold Optimization


In [9]:
from sklearn.model_selection import cross_val_predict, cross_validate

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [10]:

svm_clf = SVC(kernel='rbf', C=10, gamma=0.1, random_state=42)
mlp_clf = MLPClassifier(max_iter=10_000, hidden_layer_sizes=[10,10,10], random_state=43)
xgb_clf = XGBClassifier(max_depth=5, n_estimators=40)
lgb_clf = LGBMClassifier(max_depth=8, n_estimators=90)
cat_clf = CatBoostClassifier(cat_features=cat_indices, max_depth=6, n_estimators=1000)

In [11]:
from sklearn.metrics import (
    precision_recall_curve, recall_score, precision_score, f1_score,
    average_precision_score, roc_auc_score, matthews_corrcoef
)
from pprint import pprint

# function to evaluate a classifier with 5-fold CV with "thereshold optimization".
def run_model(clf, X_train, y_train): 
    res = cross_validate(clf, X_train, y_train, cv=5, scoring=('average_precision','roc_auc',  'recall', 'precision', 'f1', 'matthews_corrcoef'))

    auc = res['test_roc_auc'].mean()
    ap = res['test_average_precision'].mean()
    recall_no = res['test_recall'].mean()
    preci_no = res['test_precision'].mean()
    f1_no = res['test_f1'].mean()
    mcc_no = res['test_matthews_corrcoef'].mean()


    # Threshold optimization
    if (type(clf).__name__ == 'SVC'):
            y_score = cross_val_predict(clf, X_train, y_train, cv=5, method='decision_function')
    else: 
        y_score = cross_val_predict(clf, X_train, y_train, cv=5, method='predict_proba')[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_train, y_score)

    # Maximizing F1-score
    f1_val = (2 * precision * recall) / (precision + recall)
    f1_val[np.isinf(f1_val)] = np.nan

    i = np.nanargmax(f1_val)
    best_thre = thresholds[i]

    y_pred = (y_score >= best_thre)
    recall_opt = recall_score(y_train, y_pred)
    preci_opt = precision_score(y_train, y_pred)
    f1_opt = f1_score(y_train, y_pred)
    mcc_opt = matthews_corrcoef(y_train, y_pred)

    res = {'ap': ap, 'auc': auc, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc': mcc_no, 'recall(opt)': recall_opt, 'precision(opt)': preci_opt,
           'f1(opt)': f1_opt, 'mcc(opt)': mcc_opt, 'threshold': best_thre}
    return res

Run the evaluation and optimize the threshold for each model.

In [12]:
names = ['svm', 'mlp', 'lgb', 'xgb', 'cat']

cvres_svm = {}
cvres_mlp = {}
cvres_xgb = {}
cvres_lgb = {}
cvres_cat = {}

for name in names:
    clf = globals()[f'{name}_clf']
    if (name == 'cat'):
        globals()[f'cvres_{name}'] = run_model(clf, X_cat, y)
    elif (name in ['xgb', 'lgb']):
        globals()[f'cvres_{name}'] = run_model(clf, X_xgb, y)
    else:
        globals()[f'cvres_{name}'] = run_model(clf, X, y)
    

[LightGBM] [Info] Number of positive: 1051, number of negative: 19273
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000972 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 476
[LightGBM] [Info] Number of data points in the train set: 20324, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.051712 -> initscore=-2.908963
[LightGBM] [Info] Start training from score -2.908963
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 1051, number of negative: 19274
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000756 seconds.
You can set `force_row_wise=true` to remove t

Results of 5-fold CV on the training set.

In [13]:
for name in names:
    clf = globals()[f'{name}_clf']
    print("model: ", type(clf).__name__)
    pprint(globals()[f'cvres_{name}'], sort_dicts=False)
    print()

model:  SVC
{'ap': 0.42667644170275015,
 'auc': 0.8325963411845049,
 'recall': 0.22602385858996316,
 'precision': 0.7055797670820414,
 'f1': 0.3415870025651863,
 'mcc': 0.38252824140338093,
 'recall(opt)': 0.4703196347031963,
 'precision(opt)': 0.4520848573518654,
 'f1(opt)': 0.46102200671391275,
 'mcc(opt)': 0.43110605265982493,
 'threshold': -0.8283468878491065}

model:  MLPClassifier
{'ap': 0.44524359290711935,
 'auc': 0.8899325033042628,
 'recall': 0.2747365976838011,
 'precision': 0.6116501869342779,
 'f1': 0.3789116879575873,
 'mcc': 0.38975159935867343,
 'recall(opt)': 0.5213089802130898,
 'precision(opt)': 0.4327226784586229,
 'f1(opt)': 0.47290300310666206,
 'mcc(opt)': 0.44347829520096027,
 'threshold': 0.20930689241559444}

model:  LGBMClassifier
{'ap': 0.4845106245405578,
 'auc': 0.898669679093165,
 'recall': 0.28005979160015093,
 'precision': 0.6723802427369929,
 'f1': 0.3948793843129203,
 'mcc': 0.41541092164028404,
 'recall(opt)': 0.5449010654490106,
 'precision(opt)': 0

## Model Training and Test

In [14]:
# Now, we train the models on the training set
svm_clf.fit(X, y)
mlp_clf.fit(X, y)
xgb_clf.fit(X_xgb, y)
lgb_clf.fit(X_xgb, y)
cat_clf.fit(X_cat, y, cat_features=cat_indices)

[LightGBM] [Info] Number of positive: 1314, number of negative: 24092
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000849 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 478
[LightGBM] [Info] Number of data points in the train set: 25406, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.051720 -> initscore=-2.908804
[LightGBM] [Info] Start training from score -2.908804
Learning rate set to 0.041005
0:	learn: 0.6263778	total: 110ms	remaining: 1m 49s
1:	learn: 0.5705002	total: 197ms	remaining: 1m 38s
2:	learn: 0.5159115	total: 299ms	remaining: 1m 39s
3:	learn: 0.4696864	total: 397ms	remaining: 1m 38s
4:	learn: 0.4308835	total: 495ms	remaining: 1m 38s
5:	learn: 0.3955065	total: 602ms	remaining: 1m 39s
6:	learn: 0.3721400	total: 697ms	remaining: 1m 38s
7:	learn: 0.3492917	total: 794ms	remaining: 1m 38s
8:	lea

In [15]:
import joblib

names = ['svm', 'mlp', 'xgb', 'lgb', 'cat']
#names = ['svm', 'mlp', 'cat']

for name in names:
    if (name in ['xgb', 'lgb', 'cat']):
        model_dict = {
        'model': globals()[f'{name}_clf'],
        'pipeline': cat_pipeline,
        'result': globals()[f'cvres_{name}']
        }
    else:
        model_dict = {
        'model': globals()[f'{name}_clf'],
        'pipeline': full_pipeline,
        'result': globals()[f'cvres_{name}']
        }
    joblib.dump(model_dict, f'models/base_{name}_1_2.pkl')


In [16]:
test_set = pd.read_csv(f"datasets/test_set_{seed}_t_2.csv")

# Preprocessing test set
X_te = full_pipeline.transform(test_set)
label_encoding(test_set)
X_te_xgb = cat_pipeline.transform(test_set)
X_te_cat = cat_pipeline.transform(test_set)
y_te = test_set['is_drop'].copy()

X_te = pd.DataFrame(X_te, columns=full_pipeline.get_feature_names_out())
X_te_xgb = pd.DataFrame(X_te_xgb, columns=cat_pipeline.get_feature_names_out())
X_te_cat = pd.DataFrame(X_te_cat, columns=cat_pipeline.get_feature_names_out())

# Typesetting for LGBM
X_te_xgb[num_cols] = X_te_xgb[num_cols].astype(float)
X_te_xgb[cat_cols] = X_te_xgb[cat_cols].astype(int)

X_te_cat[num_cols] = X_te_cat[num_cols].astype(float)
X_te_cat[cat_cols] = X_te_cat[cat_cols].astype(int)

In [17]:
X_te_cat.rename(columns={'num__grade': 'year', 'num__gpa_last_seme': 'gpa_last', 'num__credits_last_seme': 'credits_last',
                  'num__credits_tot': 'credits_tot', 'num__n_seme': 'n_semesters', 'num__years_since': 'years_since',
                  'cat__semester': 'release', 'cat__sex': 'sex', 'cat__adm_unit': 'adm_type', 'cat__nation': 'nation',
                  'cat__in_capa': 'in_capa', 'cat__college': 'college', 'cat__leave': 'status'}, inplace=True)

In [18]:
# function to evaluate the performance with test set and given threshold

def test_model(clf, X_test, y_test, thre): 
    if (type(clf).__name__ == 'SVC'):
        y_score = clf.decision_function(X_test)
    else: y_score = clf.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_score)
    ap = average_precision_score(y_test, y_score)

    if (type(clf).__name__ == 'SVC'):
        y_pred = (y_score >= 0)
    else: y_pred = (y_score >= 0.5)
    recall_no = recall_score(y_test, y_pred)
    preci_no = precision_score(y_test, y_pred)
    f1_no = f1_score(y_test, y_pred)
    mcc_no = matthews_corrcoef(y_test, y_pred)

    # Optimizing threshold by maximizing F1-score

    y_pred = (y_score >= thre)
    recall_opt = recall_score(y_test, y_pred)
    preci_opt = precision_score(y_test, y_pred)
    f1_opt = f1_score(y_test, y_pred)
    mcc_opt = matthews_corrcoef(y_test, y_pred)


    res = {'auc': auc, 'ap': ap, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc_no': mcc_no, 'recall(opt)': recall_opt, 
           'precision(opt)': preci_opt, 'f1(opt)': f1_opt, 'mcc(opt)': mcc_opt,
           'threshold': thre}
    print("model: ", type(clf).__name__)
    pprint(res, sort_dicts=False)
    print()
    return res

In [19]:
# Now we test the six models with the test set and given thresholds.

test_svm = {}
test_mlp = {}
test_xgb = {}
test_lgb = {}
test_cat = {}

test_svm = test_model(svm_clf, X_te, y_te, cvres_svm['threshold'])
test_mlp = test_model(mlp_clf, X_te, y_te, cvres_mlp['threshold'])
test_lgb = test_model(lgb_clf, X_te_xgb, y_te, cvres_lgb['threshold'])
test_xgb = test_model(xgb_clf, X_te_xgb, y_te, cvres_xgb['threshold'])
test_cat = test_model(cat_clf, X_te_cat, y_te, cvres_cat['threshold'])


model:  SVC
{'auc': 0.8123120100295999,
 'ap': 0.4127980171636751,
 'recall': 0.2169811320754717,
 'precision': 0.7263157894736842,
 'f1': 0.3341404358353511,
 'mcc_no': 0.3821309233275488,
 'recall(opt)': 0.44339622641509435,
 'precision(opt)': 0.4533762057877814,
 'f1(opt)': 0.4483306836248013,
 'mcc(opt)': 0.4196783850467924,
 'threshold': -0.8283468878491065}

model:  MLPClassifier
{'auc': 0.8958287164921371,
 'ap': 0.43097862845190577,
 'recall': 0.2672955974842767,
 'precision': 0.625,
 'f1': 0.3744493392070485,
 'mcc_no': 0.3900011444457333,
 'recall(opt)': 0.4937106918238994,
 'precision(opt)': 0.4472934472934473,
 'f1(opt)': 0.4693572496263079,
 'mcc(opt)': 0.4405867716376551,
 'threshold': 0.20930689241559444}

model:  LGBMClassifier
{'auc': 0.9041515156244311,
 'ap': 0.46524859701109833,
 'recall': 0.2893081761006289,
 'precision': 0.6433566433566433,
 'f1': 0.39913232104121477,
 'mcc_no': 0.4129106648665507,
 'recall(opt)': 0.5188679245283019,
 'precision(opt)': 0.426356589